## Bronze → Silver (Delta Live Tables Pipeline)

This notebook defines **managed streaming tables** using Delta Live Tables.
It must be attached to a **DLT pipeline** and will not run as a standard Spark job.


### `Lakeflow Declarative Pipelines` / `Delta Live Tables`
A managed system in Databricks where you define what tables you want, and it automatically builds, updates, and monitors the data pipeline for you.

%md
> **Note:**  
> The below code will not run directly on the cluster.  
> It must be executed through a **Delta Live Tables pipeline**.


In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# Reads streaming data from the Bronze Delta table
# and creates a Silver table managed by Delta Live Tables.
@dlt.table(
    name="staging_bookings"
)
def staging_bookings():
    df = spark.readStream.format("delta") \
        .load("/Volumes/flightsproj/bronze/bronzevolume/bookings/data")
    return df

# This will laod the table incrementally.

In [0]:
@dlt.View(name="transform_bookings")

def transform_bookings():
    # Spark allow to read streaming data from the Bronze Delta table wihtout even running it at time of devlopment
    df = spark.readStream.table(staging_bookings)

    # Transform code
    df = df.withColumn('amount', col('amount').cast(DoubleType()))\
           .withColumn('booking_date', to_date(col('booking_date')))\
           .drop("_rescued_data")\
           .withColumn('modified_date', current_timestamp())
    return df

| Decorator                 | Behavior                     |
| ------------------------- | ---------------------------- |
| `@dlt.expect_all`         | Log violations only          |
| `@dlt.expect_all_or_drop` | Drop bad rows                |
| `@dlt.expect_all_or_fail` | Stop pipeline if any bad row |


Lets use @dlt.expect_all

In [0]:
rules = {
    'rule1' : 'BOOKING_ID IS NOT NULL',
    'rule2' : 'AIRPORT_ID IS NOT NULL',
    'rule3' : 'FLIGHT_ID IS NOT NULL',
    'rule4' : 'PASSENGER_ID IS NOT NULL'
}

In [0]:
# Writing the above data to streaming table

@dlt.table(name="silver_bookings")

@dlt.expect_all_or_drop(rules) # Adding expectations all rules should be satisfied or dropped
def silver_bookings():
    df = dlt.read(transform_bookings)
    return df